# State Health Rankings 2024 Visualization Final

### Introduction

For this project, I used state-level health data from the 2024 United Health Foundation’s Annual Report. After some initial cleaning and reshaping of the data, I selected a focused set of metrics that included overall health scores, economic indicators, access to care, and several health outcomes such as obesity, depression, and diabetes. These indicators were chosen because they provide a well-rounded view of both the health status and the socioeconomic context of each U.S. state.

The main goals of this visualization project were to understand how health outcomes vary across different states, identify possible relationships between economic hardship and public health conditions, and highlight states that might be under-resourced. Specifically, I was looking for patterns that could help explain disparities in health and funding, and explore different ways to visualize the data to provide to shareholders a visualization of the data.

To meet these goals, I completed several key tasks. For the first task, I compared the overall health scores of all 50 states to see where they rank. Then, I created visualizations to explore how economic hardship is related to health outcomes, like obesity. Finally, I worked to identify states that not only have poor health outcomes, but also receive less public health funding, which could make them especially vulnerable to lower rankings.

#### Data
**The csv data was downloaded from [America's Health Rankings 2024](https://www.americashealthrankings.org/learn/reports/2024-annual-report).**
The data had 1,315 data columns, further grouping the main categories I used by male/female, race, and age for each of the 50 states. The PDF report breaks down on individuals pages the findings of each state with plenty of links where the data was retrieved. I, for one, believed my beloved state of Texas would have made the top half prior to creating the visuals or that John Denver spoke heavenly about West Virginia for a reason. And then I, personally, wanted to visualize more.


In [1]:
import sys
import pandas as pd
from IPython.display import display

data = pd.read_csv(
    "2024-annual-report-report-data-all-states.csv",
    encoding="windows-1252",
    low_memory=False
)

reshaped = data.pivot_table(
    index='State',
    columns='Measure',
    values='Value'
).reset_index()

reshaped.columns.name = None 

for col in reshaped.columns:
    if col != 'State':
        reshaped[col] = pd.to_numeric(reshaped[col], errors='coerce')


In [2]:
import sys
from IPython.display import display
cols_to_keep = ["State", "Overall", "Total Population", "Social and Economic Factors", "Homicide", "Public Health Funding", "Economic Hardship Index", "Per Capita Income", "Poverty", "Unemployment", "High School Graduation", "Physical Environment", "Air Pollution", "Housing Cost Burden", "Avoided Care Due to Cost", "Uninsured", "Exercise", "Cannabis Use", "Suicide", "Cancer", "Depression", "Diabetes", "High Blood Pressure", "High Cholesterol",  "Obesity", ]
selected = reshaped[cols_to_keep]
selected = selected[selected["State"].str.upper() != "ALL"]
selected = selected[selected["State"] != "DC"]


## Task 1a: Compare Overall Health Scores Across States and Regions
**Why?** To identify which states are performing well or poorly in terms of overall health scores.
**How?** By creating a horizontal bar chart that ranks states by their `Overall` health score. A horizontal bar graph was chosen to easily distinguish top half and bottom half.
**What?** This task explores variation and ranking among states based on a composite metric (`Overall`).
**Where?** Column: `Overall` Index: `State`
**When?** Performed early in the workflow to get a high-level overview of the dataset before exploring individual health factors.
**Who?** Data analyst or public health researcher using visualization to identify priority regions for deeper analysis or intervention.

**Task 1b: Compare Overall Health Scores Across Regions (last minute bonus visualization for the previous "Who"**


In [3]:
import altair as alt

selected["Overall_Sign"] = selected["Overall"].apply(lambda x: "Positive" if x >= 0 else "Negative")

alt.Chart(selected).mark_bar().encode(
    x=alt.X("Overall:Q", title="Overall Health Score"),
    y=alt.Y("State:N", sort='-x', title="State"),
    color=alt.Color("Overall_Sign:N",
                    scale=alt.Scale(domain=["Positive", "Negative"], range=["steelblue", "darkred"]),
                    legend=None),
    tooltip=[    alt.Tooltip("State:N"),
    alt.Tooltip("Overall:Q"),
    alt.Tooltip("Uninsured:Q", title="Uninsured (%)", format=".1f")
]
).properties(
    width=800,
    height=600,
    title="Overall Health Score by State Ranking"
)


alt.Chart(...)

In [4]:
import altair as alt
from vega_datasets import data
import sys
import traceback

try:
    # TopoJSON for US states
    states = alt.topo_feature(data.us_10m.url, feature='states')

    # State abbreviation to FIPS code
    state_abbrev_to_fips = {
        'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8, 'CT': 9, 'DE': 10,
        'FL': 12, 'GA': 13, 'HI': 15, 'ID': 16, 'IL': 17, 'IN': 18,
        'IA': 19, 'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24, 'MA': 25,
        'MI': 26, 'MN': 27, 'MS': 28, 'MO': 29, 'MT': 30, 'NE': 31, 'NV': 32,
        'NH': 33, 'NJ': 34, 'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38, 'OH': 39,
        'OK': 40, 'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46, 'TN': 47,
        'TX': 48, 'UT': 49, 'VT': 50, 'VA': 51, 'WA': 53, 'WV': 54, 'WI': 55, 'WY': 56
    }

    # Filter and map to FIPS
    df_filtered = selected[selected["State"].isin(state_abbrev_to_fips.keys())].copy()
    df_filtered["id"] = df_filtered["State"].map(state_abbrev_to_fips)

    # Build map
    heatmap = alt.Chart(states).mark_geoshape().encode(
        color=alt.Color(
            "Overall:Q",
            scale=alt.Scale(scheme="redblue"),
            legend=alt.Legend(title="Overall Health Score")
        ),
        tooltip=[
            alt.Tooltip("State:N", title="State"),
            alt.Tooltip("Overall:Q", title="Health Score", format=".1f")
        ]
    ).transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_filtered, key="id", fields=["State", "Overall"])
    ).project(
        type="albersUsa"
    ).properties(
        width=700,
        height=450,
        title="Overall Health Scores by State"
    )

    heatmap

except Exception:
    print("⚠️ Error in Cell 5:")
    traceback.print_exc(file=sys.stdout)


In [5]:
selected = selected.dropna(subset=["State", "Overall"])

import altair as alt
import pandas as pd
from vega_datasets import data
import sys

states = alt.topo_feature(data.us_10m.url, feature='states')

state_abbrev_to_fips = {
    'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8, 'CT': 9, 'DE': 10,
    'FL': 12, 'GA': 13, 'HI': 15, 'ID': 16, 'IL': 17, 'IN': 18,
    'IA': 19, 'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24, 'MA': 25,
    'MI': 26, 'MN': 27, 'MS': 28, 'MO': 29, 'MT': 30, 'NE': 31, 'NV': 32,
    'NH': 33, 'NJ': 34, 'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38, 'OH': 39,
    'OK': 40, 'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46, 'TN': 47,
    'TX': 48, 'UT': 49, 'VT': 50, 'VA': 51, 'WA': 53, 'WV': 54, 'WI': 55, 'WY': 56
}

df_filtered = selected[selected["State"].isin(state_abbrev_to_fips.keys())].copy()
df_filtered = df_filtered.dropna(subset=["Overall"])
df_filtered["id"] = df_filtered["State"].map(state_abbrev_to_fips)

def running_in_voila():
    return any("voila" in arg.lower() for arg in sys.argv)

if not running_in_voila():
    print("Map will include", df_filtered.shape[0], "states.")
    
heatmap = alt.Chart(states).mark_geoshape().encode(
    color=alt.Color(
        "Overall:Q",
        scale=alt.Scale(scheme="redblue"),
        legend=alt.Legend(title="Overall Health Score")
    ),
    tooltip=[
        alt.Tooltip("State:N", title="State"),
        alt.Tooltip("Overall:Q", title="Health Score", format=".1f")
    ]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df_filtered, key="id", fields=["State", "Overall"])
).project(
    type="albersUsa"
).properties(
    width=700,
    height=450,
    title="Overall Health Scores by State"
)

heatmap


Map will include 50 states.


alt.Chart(...)

## Task 2: Explore Relationships Between Economic and Health Factors
**Why?** To understand how economic hardship correlates with health outcomes, like obesity, unisured rates, exercise.
**How?** Use a scatter plot comparing `Economic Hardship Index` with `Obesity`, `Unisured`, `High Blood Pressure`, ` High Cholesterol`, `Diabetes`, or `Exercise`.
**What?** This task examines correlation and outliers in two-variable comparisons. The correlation regression line can be checked to visualize any positive or negative correlation. Included is the tooltip to hover and view individual states and scores.
**Where?** Columns: `Econimc Hardship Index`, `Obesity`, `Uninsured`, etc Index: `State`
**When?** Performed after initial exploration to generate hypotheses about cause-and-effect or clustering of outcomes.
**Who?** Policy analyst or public health planner evaluating where socioeconomic policy changes may have the greatest health impacts.

In [6]:
import altair as alt
import pandas as pd

melted = selected.melt(
    id_vars=["State", "Economic Hardship Index"],
    value_vars=[
        "Avoided Care Due to Cost", "Cancer", "Diabetes",  "Exercise", 
        "High Blood Pressure", "High Cholesterol", "Obesity",
        "Uninsured" 
    ],
    var_name="Health Metric",
    value_name="Value"
)

health_metric_param = alt.param(
    name="health_metric",
    bind=alt.binding_select(
        options=[
            "Avoided Care Due to Cost", "Cancer", "Diabetes",  "Exercise", 
            "High Blood Pressure", "High Cholesterol", "Obesity",
            "Uninsured"  
        ],
        name="Health Metric: "
    ),
    value="Avoided Care Due to Cost"
)

show_regression = alt.param(
    name="show_regression",
    bind=alt.binding_checkbox(name="Show Correlation Line"),
    value=False
)

points = alt.Chart(melted).mark_circle(size=100).encode(
    x=alt.X("Economic Hardship Index:Q", title="Economic Hardship Index"),
    y=alt.Y("Value:Q", title="Health Metric (%)"),
    color=alt.value("steelblue"),
    tooltip=[
        alt.Tooltip("State:N"),
        alt.Tooltip("Economic Hardship Index:Q", format=".1f"),
        alt.Tooltip("Value:Q", format=".1f"),
        alt.Tooltip("Health Metric:N")
    ]
).transform_filter(
    alt.datum["Health Metric"] == health_metric_param
)

trendline = alt.Chart(melted).transform_filter(
    (alt.datum["Health Metric"] == health_metric_param) & show_regression
).transform_regression(
    "Economic Hardship Index", "Value", method="linear"
).mark_line(color="darkred", strokeDash=[5, 5]).encode(
    x="Economic Hardship Index:Q",
    y="Value:Q"
)


chart = (points + trendline).add_params(
    health_metric_param,
    show_regression
).properties(
    width=500,
    height=350,
    title="Economic Hardship vs. Selected Health Metric (with Toggleable Trend Line)"
)

chart


alt.LayerChart(...)

## Task 3: Identify States with Health Resource Deficits
**Why?** To find states where poor outcomes are linked with low public health funding or access to care.
**How?** Use a combination of heatscatter plot and filtering to isolate states with low `Public Health Funding` and high `Avoided Care Due to Cost`. All values were normalized on a scale of 0-1, with Public Health Funding and Exercise reversed. This was done so lower value in these categories would be similar to the other lower values in the health metrics. It includes a dropdown to visualize where a state's value falls in a few different health metrics. 
**What?** This task focuses on identifying the thresholds, bottlenecks, and gaps in resource allocation.
**Where?** Columns: `Public Health Funding`, `Avoided Care Due to Cost`, `Uninsured`
**When?** Mid-to-late analysis phases, guiding recommendations or mapping potential interventions.
**Who?** Health equity advocate or nonprofit stategist aiming to prioritize underserved areas.

In [7]:
import altair as alt
import pandas as pd

df_norm = selected.copy()

metrics = [
    "Public Health Funding", 
    "Avoided Care Due to Cost",
    "Uninsured",
    "Economic Hardship Index",
    "Poverty",
    "Cannabis Use",
    "Obesity",
    "High Blood Pressure",
    "High Cholesterol",
    "Diabetes",
    "Exercise",
    "Suicide",
    "Homicide"    
]

for metric in metrics:
    min_val = df_norm[metric].min()
    max_val = df_norm[metric].max()
    df_norm[f"{metric} Norm"] = (df_norm[metric] - min_val) / (max_val - min_val)

df_norm["Public Health Funding Norm"] = 1 - df_norm["Public Health Funding Norm"]
df_norm["Exercise Norm"] = 1 - df_norm["Exercise Norm"]


norm_cols = [f"{m} Norm" for m in metrics]
melted = df_norm.melt(
    id_vars=["State"],
    value_vars=norm_cols,
    var_name="Metric",
    value_name="Value"
)

melted["Metric"] = melted["Metric"].str.replace(" Norm", "", regex=False)
melted.loc[melted["Metric"] == "Public Health Funding", "Metric"] = "Public Health Funding (Reversed)"
melted.loc[melted["Metric"] == "Exercise", "Metric"] = "Exercise (Reversed)"


state_options = ["All"] + sorted(df_norm["State"].unique())
state_dropdown = alt.binding_select(options=state_options, name="Select State: ")
state_param = alt.param(name="state_param", bind=state_dropdown, value="All")

base = alt.Chart(melted).mark_circle(size=100, opacity=0.2).encode(
    x=alt.X("Metric:N", title="Health Metric", sort=None),
    y=alt.Y("Value:Q", title="Normalized Value", scale=alt.Scale(domain=[0, 1])),
    color=alt.value("lightgray"),
    tooltip=[
        alt.Tooltip("State:N"),
        alt.Tooltip("Metric:N"),
        alt.Tooltip("Value:Q", format=".2f")
    ]
).transform_filter(
    (state_param == "All") | (alt.datum.State != state_param)
)

highlight = alt.Chart(melted).mark_circle(size=300).encode(
    x=alt.X("Metric:N", title="Health Metric", sort=None),
    y=alt.Y("Value:Q", title="Normalized Value", scale=alt.Scale(domain=[0, 1])),
    color=alt.Color(
        "Value:Q",
        scale=alt.Scale(scheme="turbo", domain=[0, 1]),
        legend=alt.Legend(title="Normalized Value")
    ),
    tooltip=[
        alt.Tooltip("State:N"),
        alt.Tooltip("Metric:N"),
        alt.Tooltip("Value:Q", format=".2f")
    ]
).transform_filter(
    alt.datum.State == state_param
)

heat_scatter = (base + highlight).add_params(
    state_param
).properties(
    width=700,
    height=400,
    title="Normalized Health Metrics Heat Scatterplot (0–1 Scale)"
)

heat_scatter


alt.LayerChart(...)

### Design Summary and Justification
All of the visualizations were created using Altair for interactive features. It allowed me to quickly experiment with design ideas and incorporate interactivity, such as tooltips and dropdown menus, without making the visuals too complex.

Color was used strategically throughout the charts. For example, in the bar chart, color highlights whether a state is performing above or below average. In the scatterplots and heatmaps, color helps to emphasize variation in health metrics. This makes it easier for users to spot trends and outliers at a glance.

The layout of each chart was designed with usability in mind. Horizontal bar charts were used so top half and bottom half would be readable. Scatterplots were included to help viewers visually assess correlations between economic conditions and health outcomes. Heatmaps and dot plots allowed for quick comparisons across multiple dimensions.

Overall, I focused on keeping the design clean and not overwhelming the viewer. Tooltips were added to provide helpful context without adding visual clutter. Sort options and interactivity made the charts more engaging and helped users explore the data in ways that were relevant to their own questions.

### Evaluation Summary
To evaluate the effectiveness of my visualizations, I shared them with three people who had some basic familiarity with data, two mechanical engineer  friends and my mother. Each completed four short questions designed to reflect real analysis scenarios, followed by a brief interview to gather their feedback. I provided them a link to the interactive charts using Google Colab or Jupyter.

**Questions**
1) Which three states are ranked the lowest? 
2) Which area of the United States seems to be considered healthy based on their overall scores?
3) Does Economic Hardship have a positive or negative correlation with Avoided Care due to Cost?
4) From the dropdown menu, are most of Colorado's health markers bad/towards the top or good/near the bottom?

I tracked how long it took them to answer, how confident they were, and then asked a few follow-up questions. The goal was to see how easily they could understand the charts, whether the visuals supported their reasoning, and if anything was confusing or difficult.

The results were mostly positive. All three participants were able to complete the tasks. They said the bar chart and heatmap were the most straightforward to read. The heat scatterplot led to more questions and discussion, which showed it was thought-provoking but slightly harder to interpret.

### Synthesis of Findings

Through this project, I found that several aspects of my design worked well. The use of color and sorting helped people quickly notice important patterns, such as which states were struggling or which had better funding. The interactive tooltips were also appreciated, as they gave people access to more detailed information without making the chart look too busy.

Participants generally found the charts clear and helpful for answering the questions. However, there are also areas I would like to improve. For example, adding filters to allow users to explore by region or specific health topics could make the charts more powerful. Another idea is to combine multiple charts into a single dashboard view, which would make it easier to compare metrics side by side.

Throughout the project, my plan evolved based on what I learned during the process. I initially only planned to make a bar chart and scatterplot, but added a heatmap after realizing it helped address one of my key goals. I also adjusted the tooltips and formatting based on early user feedback to make them clearer. In the evaluation phase, I decided not only to measure accuracy but also ask for people’s confidence and reflections, which gave me more insight into how effective the visuals were.

In summary, this project helped me learn a lot about designing for clarity, interactivity, user needs, and how to adapt a data analysis plan based on what the data and users reveal along the way.